# What's the weather in your favourite band lyrics?

The aim of the project is to write a script that will analyze the songs of a given artist in terms of the weather in the places mentioned in their lyrics. Having only the name of the band, we need to get the lyrics of all their songs, then look for the names of places in them, and finally check what the weather is like in the places mentioned in the lyric.

## API used

__AudioDB service__ [https://www.theaudiodb.com/api_guide.php](https://www.theaudiodb.com/api_guide.php) <br>
Since the AudioDB service was experiencing quite high public key overhead, the following restrictions were introduced:Public API Key has been changed from 1 to 2, We can only execute the query once every two seconds

__Geocode__ [https://geocode.xyz](https://geocode.xyz) for download the names of cities, villages, states<br>

__OpenWeather__ for download the weather for places mentioned in the lyrics [endpoint documentation /weather](https://openweathermap.org/current#name).


In [1]:
THEAUDIODB_KEY = '2'
GEOCODEXYZ_KEY = '619162889546444104515x21146'
OPENWEATHERMAPORG_KEY = "2e67edfea86f7ac9c274cab6ed5494ff"


## Download the song list

We can download the song list from AudioDB service when we know the ID of the discs they are on. However, we will only know the ID of the albums when we know the ID of the artist.

So we need to start by searching for the artist by name.

Function `get_band` takes a band name and returns a dictionary with the keys `id` and `name` - or `None` if the band couldn't be found.

In [2]:
import requests
from time import sleep

In [3]:
def get_band(name):
    print('Downloading band data...')
    band_details_response = requests.get(f'https://www.theaudiodb.com/api/v1/json/{THEAUDIODB_KEY}/search.php', {'s': name})
    band_details = band_details_response.json()

    sleep(2)  # required by API documentation

    if band_details['artists']:
        return {'id': band_details['artists'][0]['idArtist'], 'name': band_details['artists'][0]['strArtist']}
    else:
        return None

The above function will provide us with an artist ID that we will use to download a list of his albums.

Function `get_album_ids` take the artist's ID and return a list of album IDs - these are the only ones we need to finally retrieve the names of the artist's songs.

In [4]:
def get_album_ids(band_id):
    print('Downloading albums...')
    albums_response = requests.get(f'https://theaudiodb.com/api/v1/json/{THEAUDIODB_KEY}/album.php', {'i': band_id})
    albums = albums_response.json()

    sleep(2)  # required by API documentation

    return [album['idAlbum'] for album in albums['album']]

Now we need to write a function that can download songs from one album ID - later we will use it in a loop to query the API for all albums.

At first, we are only interested in song names. Function `get_tracks` return track title

In [5]:
def get_tracks(album_id):
    print(f'Downloading tracks from an album {album_id}...')
    tracks_response = requests.get(f'https://theaudiodb.com/api/v1/json/{THEAUDIODB_KEY}/track.php', {'m': album_id})
    tracks = tracks_response.json()

    sleep(2)  # required by API documentation

    return [track['strTrack'] for track in tracks['track']]

## Download song lyrics

Everything works fine when the lyrics are in its database. The problem begins when the text is not there - in such a situation, the API does not close the connection for a very long time, and keeps its client in suspense...

Fortunately, the `requests` library allows us to specify the maximum time we are willing to wait for a response from the server. We will use `timeout` [Documentation](https://requests.readthedocs.io/en/master/user/quickstart/#timeouts)
When time passes, the function will throw an exception instead of returning a response.

The second problem is that the band and song names are passed to the API in the address, but not in the querystring. What if a band has `?` in its name? The API will definitely misinterpret such a query. The solution is to replace all "dangerous" characters with their codes in the form `%xx` - this will be done by the `quote` function from the `urllib.parse` module.

Function `get_lyrics` returns the lyrics of the song, or `None` if it could not be found.

In [6]:
from urllib.parse import quote

def get_lyrics(band, title):
    try:
        response = requests.get(f'https://api.lyrics.ovh/v1/{quote(band)}/{quote(title)}', timeout=15)
        sleep(2)  # required by API documentation

        return response.json()['lyrics']
    except Exception:
        print(f'No lyrics for {band} - {title}')
        return None

## Places mentioned in the lyric

For download the names of cities, villages, states we will use [https://geocode.xyz](https://geocode.xyz).

API returns results in 3 different ways:
- If no place is found, the `"match"` key will not appear in the response at all.
- If only one place was found, the `"match"` key will not contain a list - instead it will immediately include a dictionary with the only place found.
- If more places are found, the `"match"` key will contain the list.

To make the function more consistent, each of these three situations should be recognized and handled accordingly

In [7]:
def get_places(text):
    response = requests.post('https://geocode.xyz', {'scantext': text, 'geoit': 'json', 'sentiment': 'analysis', 'auth': GEOCODEXYZ_KEY})
    if response.status_code == 200:
        content = response.json()
        if 'match' not in content:
            print('No places in the text')
            return []
        if type(content['match']) == list:
            return [match['location'] for match in content['match']]
        else:
            return [content['match']['location']]
    else:
        print('The query to geocode.xyz returned with status', response.status_code)
        return []

## Weather download

OpenWeather API [documentation](https://openweathermap.org/current#name) returns a response with the status `200 OK` and the weather for the given place, or the response `404 NOT FOUND` if the place was not found.

Using `response.ok` we will check which of these situations we are dealing with and return the result accordingly.

In [8]:
def get_weather(location):
    response = requests.get(f'http://api.openweathermap.org/data/2.5/weather', {'q': location, 'appid': OPENWEATHERMAPORG_KEY})
    if response.ok:
        return response.json()['weather'][0]['description']
    else:
        print('No weather found for:', location)
        return None

## Putting it all together

It's time to use all the function one by one. We'll start by asking the user for the band name. Then we will use the `get_band` function to get the name and, most importantly, the artist ID.

If the operation is successful, in `band` we will have a dictionary with the keys `"name"` and `"id"` - if not, in `band` it will be `None` and the only thing left for us to do is to inform the user that his favorite band is not found.

After successfully downloading the artist ID, we will download the IDs of all his albums, and then in the `all_tracks` variable we will collect the titles of all songs from each album.

Once the song titles are downloaded, we will query the appropriate API with the `get_lyrics` function for the lyrics. If it's not there, we'll go back to the beginning of the loop for another song. Using `not` and `continue` prevents the code from straying too far from the left edge.

Then we will download the places mentioned in the lyrics - the `get_places` function always returns a list of strings. There is no need to check whether the list is empty - if it is, the `for` loop will simply execute zero times.

In the middle of the above-mentioned loop, we will ask the `get_weather` function about the weather in a given place. The `get_weather` function returns either a weather description (of type `str`) or `None` - it is worth checking whether the result of the function is "truthy" - to skip places for which no weather was found.

## Snippet of results

Snippet of results for **Train**:
```
No lyrics for Train - For Me, It's You
Weather for SAN FRANCISCO,US: few clouds (Save Me, San Francisco - Train)
Weather for OREGON,US: overcast clouds (Save Me, San Francisco - Train)
Weather for SEATTLE,US: mist (Save Me, San Francisco - Train)
Weather for GOLDEN,US: broken clouds (Save Me, San Francisco - Train)
Weather for MARIN,US: clear sky (Save Me, San Francisco - Train)
No weather found for: ALCATRAZ,US
Weather for HIGHWAY,US: clear sky (Save Me, San Francisco - Train)
Weather for FILLMORE,US: clear sky (Save Me, San Francisco - Train)
```


In [9]:
name = input('Enter a band name: ')

band = get_band(name)
if band:
    print('Found:', band['name'])
    
    album_ids = get_album_ids(band['id'])
    
    all_tracks = []
    for aid in album_ids:
        album_tracks = get_tracks(aid)
        all_tracks.extend(album_tracks)
    
    for track_name in all_tracks:
        lyrics = get_lyrics(band['name'], track_name)
        if not lyrics:
            continue

        places = get_places(lyrics)
        
        for place in places:
            weather = get_weather(place)
            if weather:
                print(f'Weather for {place}: {weather} ({track_name} - {band["name"]})')
                
else:
    print('Not found:', name)

Enter a band name: coldplay
Found: Coldplay
No lyrics for Coldplay - Politik
No lyrics for Coldplay - In My Place
No lyrics for Coldplay - God Put a Smile Upon Your Face
No lyrics for Coldplay - The Scientist
No lyrics for Coldplay - Clocks
No lyrics for Coldplay - Daylight
No lyrics for Coldplay - Green Eyes
No lyrics for Coldplay - Warning Sign
No lyrics for Coldplay - A Whisper
No lyrics for Coldplay - A Rush of Blood to the Head
No lyrics for Coldplay - Amsterdam
No lyrics for Coldplay - Don't Panic
No lyrics for Coldplay - Shiver
No lyrics for Coldplay - Spies
No lyrics for Coldplay - Sparks
No lyrics for Coldplay - Yellow
No lyrics for Coldplay - Trouble
No lyrics for Coldplay - Parachutes
No lyrics for Coldplay - High Speed
No lyrics for Coldplay - We Never Change
No lyrics for Coldplay - Everything's Not Lost
No lyrics for Coldplay - Life is For Living
No lyrics for Coldplay - Mylo Xyloto
No lyrics for Coldplay - Hurts Like Heaven
No lyrics for Coldplay - Paradise
No lyrics for

No lyrics for Coldplay - Careful Where You Stand
No lyrics for Coldplay - Yellow
No lyrics for Coldplay - Help Is Round the Corner
No lyrics for Coldplay - Trouble
No lyrics for Coldplay - Brothers and Sisters
No lyrics for Coldplay - Always in My Head
No lyrics for Coldplay - Magic
No lyrics for Coldplay - Ink
No lyrics for Coldplay - True Love
No lyrics for Coldplay - Midnight
No lyrics for Coldplay - Another's Arms
No lyrics for Coldplay - Oceans
No lyrics for Coldplay - A Sky Full of Stars
No lyrics for Coldplay - O
No lyrics for Coldplay - Viva la vida
No lyrics for Coldplay - Life in Technicolor ii (radio edit)
No lyrics for Coldplay - Speed of Sound
No lyrics for Coldplay - In My Place
No lyrics for Coldplay - Trouble
No lyrics for Coldplay - Talk
No lyrics for Coldplay - Lovers in Japan
No lyrics for Coldplay - Shiver
No lyrics for Coldplay - Yellow
No lyrics for Coldplay - Lost!
No lyrics for Coldplay - Fix You
No lyrics for Coldplay - The Scientist
No lyrics for Coldplay - Do

No lyrics for Coldplay - Talk
No lyrics for Coldplay - The Hardest Part
No lyrics for Coldplay - What If
No lyrics for Coldplay - White Shadows
No lyrics for Coldplay - Violet Hill
No lyrics for Coldplay - Viva la vida
No lyrics for Coldplay - Lovers in Japan / Reign of Love
No lyrics for Coldplay - Lost!
No lyrics for Coldplay - Lhuna
No lyrics for Coldplay - Life in Technicolor II
No lyrics for Coldplay - Christmas Lights
No lyrics for Coldplay - Every Teardrop Is a Waterfall
No lyrics for Coldplay - Major Minus
No lyrics for Coldplay - Moving to Mars
No lyrics for Coldplay - A Head Full of Dreams
No lyrics for Coldplay - Birds
No lyrics for Coldplay - Hymn for the Weekend
No lyrics for Coldplay - Everglow
No lyrics for Coldplay - Adventure of a Lifetime
No lyrics for Coldplay - Fun
No lyrics for Coldplay - Kaleidoscope
No lyrics for Coldplay - Army of One / [X Marks the Spot]
No lyrics for Coldplay - Amazing Day
No lyrics for Coldplay - Colour Spectrum
No lyrics for Coldplay - Up&Up

No lyrics for Coldplay - Humankind
No lyrics for Coldplay - ✨
No lyrics for Coldplay - Let Somebody Go
No lyrics for Coldplay - ❤️
No lyrics for Coldplay - People of the Pride
No lyrics for Coldplay - Biutyful
No lyrics for Coldplay - My Universe
No lyrics for Coldplay - ♾
No lyrics for Coldplay - Coloratura
No lyrics for Coldplay - A Head Full of Dreams
No lyrics for Coldplay - Yellow
No lyrics for Coldplay - The Scientist
No lyrics for Coldplay - Paradise
No lyrics for Coldplay - Everglow
No lyrics for Coldplay - Midnight / Charlie Brown
No lyrics for Coldplay - Hymn for the Weekend
No lyrics for Coldplay - Fix You
No lyrics for Coldplay - Viva La Vida
No lyrics for Coldplay - Adventure of a Lifetime
No lyrics for Coldplay - Us Against the World
No lyrics for Coldplay - Something Just Like This
No lyrics for Coldplay - A Sky Full of Stars
No lyrics for Coldplay - Up&Up
No lyrics for Coldplay - Higher Power (acoustic version)
